# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 58 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.12534


extracting tarball to tmp_2211.12534...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.12547


extracting tarball to tmp_2211.12547...

...

extracting tarball to tmp_2211.13167...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.13193


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.13167/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'journal_abbrev' from 'tmp_2211.13167/journal_abbrev.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2211.13193...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12613) | **(Nearly) Model-Independent Constraints on the Neutral Hydrogen Fraction  in the Intergalactic Medium at $z\sim 5-7$ Using Dark Pixel Fractions in  Ly$α$ and Ly$β$ Forests**  |
|| Xiangyu Jin, et al. -- incl., <mark>Frederick B. Davies</mark>, <mark>Joseph F. Hennawi</mark>, <mark>Fabian Walter</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *13 pages, 6 figures, 2 tables, accepted for publication in ApJ*|
|**Abstract**| Cosmic reionization was the last major phase transition of hydrogen from neutral to highly ionized in the intergalactic medium (IGM). Current observations show that the IGM is significantly neutral at $z>7$, and largely ionized by $z\sim5.5$. However, most methods to measure the IGM neutral fraction are highly model-dependent, and are limited to when the volume-averaged neutral fraction of the IGM is either relatively low ($\bar{x}_{\rm HI} \lesssim 10^{-3}$) or close to unity ($\bar{x}_{\rm HI}\sim 1$). In particular, the neutral fraction evolution of the IGM at the critical redshift range of $z=6-7$ is poorly constrained. We present new constraints on $\bar{x}_{\rm HI}$ at $z\sim5.1-6.8$, by analyzing deep optical spectra of $53$ quasars at $5.73<z<7.09$. We derive model-independent upper limits on the neutral hydrogen fraction based on the fraction of "dark" pixels identified in the Lyman $\alpha$ (Ly$\alpha$) and Lyman $\beta$ (Ly$\beta$) forests, without any assumptions on the IGM model or the intrinsic shape of the quasar continuum. They are the first model-independent constraints on the IGM neutral hydrogen fraction at $z\sim6.2-6.8$ using quasar absorption measurements. Our results give upper limits of $\bar{x}_{\rm HI}(z=6.3) < 0.79\pm0.04$ (1$\sigma$), $\bar{x}_{\rm HI} (z=6.5) < 0.87\pm0.03$ (1$\sigma$), and $\bar{x}_{\rm HI} (z=6.7) < 0.94^{+0.06}_{-0.09}$ (1$\sigma$). The dark pixel fractions at $z>6.1$ are consistent with the redshift evolution of the neutral fraction of the IGM derived from the Planck 2018. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13146-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13146) | **iMaNGA: mock MaNGA galaxies based on IllustrisTNG-50 and MaStar SSPs.  II. the catalogue**  |
|| Lorenza Nanni, et al. -- incl., <mark>Justus Neumann</mark>, <mark>Lewis Hill</mark>, <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *arXiv admin note: text overlap with arXiv:2203.11575*|
|**Abstract**| To probe the current theory on galaxy formation and evolution, an increased synergy between observations and simulations is necessary. For this reason, in our previous paper of this series, we presented a method to mock SDSS-IV/MaNGA integral-field spectroscopic galaxy observations from cosmological simulations of galaxy formation. Here we present the resulting mock galaxy catalogue. This catalogue consists of 1,000 unique galaxies in IllustrisTNG-50 falling into the SDSS-IV/MaNGA-primary target footprint, defined in the redshift and i-band absolute magnitude space. In this paper, we describe the general characteristics of the catalogue, in terms of morphology, kinematics, and stellar population properties. We also investigate our ability to recover the galaxy characteristics, as given by the simulations, analysing the synthetic spectra. We show that the `intrinsic' and recovered kinematics, age and metallicity are consistent within 1${\sigma}$, with residuals over all tassels ($\sim 8$ million) consistent with $0$ at the $68\%$ confidence level. We also compare `intrinsic' and recovered star formation histories, finding a close resemblance. Therefore, our mocking and spectral fitting processes do not distort intrinsic galaxy properties, hence we can use these results for scientific analysis. In the next papers of this series, we will present a comprehensive comparison and scientific analysis of TNG50 simulations with MaNGA observational results. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12576-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12576) | **A Swing of the Pendulum: The Chemodynamics of the Local Stellar Halo  Indicate Contributions from Several Radial Merger Events**  |
|| Thomas Donlon II, <mark>Heidi Jo Newberg</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| **|
|**Abstract**| We find that the chemical abundances and dynamics of APOGEE and GALAH stars in the local stellar halo are inconsistent with a scenario in which the inner halo is primarily composed of debris from a single, massive, ancient merger event, as has been proposed to explain the Gaia-Enceladus/Gaia Sausage (GSE) structure. The data contains trends of chemical composition with energy which are opposite to expectations for a single massive, ancient merger event, and multiple chemical evolution paths with distinct dynamics are present. We use a Bayesian Gaussian mixture model regression algorithm to characterize the local stellar halo, and find that the data is best fit by a model with four components. We interpret these components as the VRM, Cronus, Nereus, and Thamnos; however, Nereus and Thamnos likely represent more than one accretion event because the chemical abundance distributions of their member stars contain many peaks. Although the Cronus and Thamnos components have different dynamics, their chemical abundances suggest they may be related. We show that the distinct low and high alpha halo populations from Nissen & Schuster (2010) are explained by VRM and Cronus stars, as well as some in-situ stars. Because the local stellar halo contains multiple substructures, different popular methods of selecting GSE stars will actually select different mixtures of these substructures, which may change the apparent chemodynamic properties of the selected stars. We also find that the Splash stars in the solar region are shifted to higher v_phi and slightly lower [Fe/H] than previously reported. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13193-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13193) | **Taking a break: paused accretion in the symbiotic binary RT Cru**  |
|| A. Pujol, et al. -- incl., <mark>F. M. Walter</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *Accepted in A&A*|
|**Abstract**| Symbiotic binaries sometimes hide their symbiotic nature for significant periods of time. There is mounting observational evidence that in those symbiotics that are powered solely by accretion of red-giant's wind material onto a white dwarf, without any quasi-steady shell burning on the surface of the white dwarf, the characteristic emission lines in the optical spectrum can vanish, leaving the semblance of an isolated red giant spectrum. Here we present compelling evidence that this disappearance of optical emission lines from the spectrum of RT Cru during 2019 was due to a decrease in the accretion rate, which we derive by modeling the X-ray spectrum. This drop in accretion rate leads to a lower flux of ionizing photons and thus to faint/absent photoionization emission lines in the optical spectrum. We observed the white dwarf symbiotic RT Cru with XMM-Newton and Swift in X-rays and UV and collected ground-based optical spectra and photometry over the last 33 years. This long-term coverage shows that during most of the year 2019, the accretion rate onto the white dwarf was so low, $\dot{M}= (3.2\pm 0.06)\, \times$10$^{-11}$ $M_{\odot}$ yr$^{-1}$ (d/2.52 kpc)$^2$, that the historically detected hard X-ray emission almost vanished, the UV flux faded by roughly 5 magnitudes, the $U$, $B$ and $V$ flickering amplitude decreased, and the Balmer lines virtually disappeared from January through March 2019. Long-lasting low-accretion episodes as the one reported here may hamper the chances of RT Cru experiencing nova-type outburst despite the high-mass of the accreting white dwarf. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12666-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12666) | **Inferring Maps of the Sun's Far-side Unsigned Magnetic Flux from  Far-side Helioseismic Images using Machine Learning Techniques**  |
|| Ruizhu Chen, et al. -- incl., <mark>Junwei Zhao</mark>, <mark>Shea Hess Webber</mark>, <mark>Yang Liu</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *Accepted by ApJ*|
|**Abstract**| Accurate modeling of the Sun's coronal magnetic field and solar wind structures require inputs of the solar global magnetic field, including both the near and far sides, but the Sun's far-side magnetic field cannot be directly observed. However, the Sun's far-side active regions are routinely monitored by helioseismic imaging methods, which only require continuous near-side observations. It is therefore both feasible and useful to estimate the far-side magnetic-flux maps using the far-side helioseismic images despite their relatively low spatial resolution and large uncertainties. In this work, we train two machine-learning models to achieve this goal. The first machine-learning training pairs simultaneous SDO/HMI-observed magnetic-flux maps and SDO/AIA-observed EUV 304$\r{A}$ images, and the resulting model can convert 304$\r{A}$ images into magnetic-flux maps. This model is then applied on the STEREO/EUVI-observed far-side 304$\r{A}$ images, available for about 4.3 years, for the far-side magnetic-flux maps. These EUV-converted magnetic-flux maps are then paired with simultaneous far-side helioseismic images for a second machine-learning training, and the resulting model can convert far-side helioseismic images into magnetic-flux maps. These helioseismically derived far-side magnetic-flux maps, despite their limitations in spatial resolution and accuracy, can be routinely available on a daily basis, providing useful magnetic information on the Sun's far side using only the near-side observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12684-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12684) | **Isochrone fitting of Galactic globular clusters -- IV. NGC6362 and  NGC6723**  |
|| George A. Gontcharov, et al. -- incl., <mark>Jae-Woo Lee</mark>, <mark>Noah Hebdon</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *Accepted to be published in MNRAS, 20 pages, 3 online tables*|
|**Abstract**| We present new isochrone fits to the colour-magnitude diagrams of the Galactic globular clusters NGC\,6362 and NGC\,6723. We utilize 22 and 26 photometric filters for NGC\,6362 and NGC\,6723, respectively, from the ultraviolet to mid-infrared using data sets from {\it HST}, {\it Gaia}, unWISE, and other photometric sources. We use models and isochrones from the Dartmouth Stellar Evolution Database (DSED) and Bag of Stellar Tracks and Isochrones (BaSTI) for $\alpha$-enhanced [$\alpha$/Fe]$=+0.4$ and different helium abundances. The metallicities [Fe/H]$=-1.04\pm0.07$ and $-1.09\pm0.06$ are derived from the red giant branch slopes in our fitting for NGC\,6362 and NGC\,6723, respectively. They agree with spectroscopic estimates from the literature. We find a differential reddening up to $\Delta E(B-V)=0.13$ mag in the NGC\,6723 field due to the adjacent Corona Australis cloud complex. We derive the following for NGC\,6362 and NGC\,6723, respectively: distances $7.75\pm0.03\pm0.15$ (statistic and systematic error) and $8.15\pm0.04\pm0.15$ kpc; ages $12.0\pm0.1\pm0.8$ and $12.4\pm0.1\pm0.8$ Gyr; extinctions $A_\mathrm{V}=0.19\pm0.04\pm0.06$ and $0.24\pm0.03\pm0.06$ mag; reddenings $E(B-V)=0.056\pm0.01\pm0.02$ and $0.068\pm0.01\pm0.02$ mag. DSED provides systematically lower [Fe/H] and higher reddenings than BaSTI. However, the models agree in their relative estimates: NGC\,6723 is $0.44\pm0.04$ kpc further, $0.5\pm0.1$ Gyr older, $\Delta E(B-V)=0.007\pm0.002$ more reddened, and with $0.05\pm0.01$ dex lower [Fe/H] than NGC\,6362. The lower metallicity and greater age of NGC\,6723 with respect to NGC\,6362 explain their horizontal branch morphology difference. This confirms age as the second parameter for these clusters. We provide lists of the cluster members from the {\it Gaia} Data Release 3. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xb9 in position 9392: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12557-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12557) | **The CO$_2$ Profile and Analytical Model for the Pioneer Venus Large  Probe Neutral Mass Spectrometer**  |
|| Rakesh Mogul, <mark>Sanjay S. Limaye</mark>, Michael. J. Way |
|*Appeared on*| *2022-11-24*|
|*Comments*| *Accepted, Nov. 2022*|
|**Abstract**| We present a significantly updated CO$_2$ altitude profile for Venus (64.2-0.9 km) and provide support for a potential deep lower atmospheric haze of particles (17 km and lower). We extracted this information by developing a new analytical model for mass spectra obtained by the Pioneer Venus Large Probe (PVLP) Neutral Mass Spectrometer (LNMS). Our model accounts for changes in LNMS configuration and output during descent and enables the disentanglement of isobaric species via a data fitting routine that adjusts for mass-dependent changes in peak shape. The model yields CO$_2$ in units of density (kg m-3), isotope ratios for $^{13}$C/$^{12}$C and $^{18}$O/$^{16}$O, and 14 measures of CO$_2$ density across 55.4-0.9 km, which represents the most complete altitude profile for CO$_2$ at 60 km towards the surface to date. The CO$_2$ density profile is also consistent with the pressure, temperature, and volumetric gas measurements from the PVLP and VeNeRa spacecraft. Nominal and low-noise operations for the LNMS mass analyzer are supported by the behaviors (e.g., ionization yields, fragmentation yields, and peak shapes) of several internal standards (e.g., CH$^{3+}$, CH$^{4+}$, $^{40}$Ar$^+$, $^{136}$Xe$^{2+}$, and $^{136}$Xe$^+$), which were tracked across the descent. Lastly, our review of the CO$_2$ profile and LNMS spectra reveals hitherto unreported partial and rapidly clearing clogs of the inlet in the lower atmosphere, along with several ensuing data spikes at multiple masses. Together, these observations suggest that atmospheric intake was impacted by particles at 17 km (and lower) and that rapid particle degradation at the inlet yielded a temporary influx of mass signals into the LNMS. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.12843-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.12843) | **Discovery and description of two young open clusters in the primordial  group of NGC 6871**  |
|| Juan Casado, <mark>Yasser Hendy</mark> |
|*Appeared on*| *2022-11-24*|
|*Comments*| *21 pages, 13 figures, 4 tables*|
|**Abstract**| A primordial group of open clusters containing NGC 6871 is confirmed and described through Gaia DR3 data and the previous literature. It is a star-forming complex containing at least six young OCs, including Teutsch 8, FSR 198 and Biurakan 2. Two nearby OCs (Casado 82 and Casado-Hendy 1) are newly identified and studied in detail and found to be also members of the cited group. The parameters of the components are sufficiently similar to postulate the case of at least six clusters born from a single GMC. None of the cluster pairs of the group seems to be an authentic binary cluster, with the possible exception of the candidate pair Teutsch 8/FSR 198. Instead, NGC 6871 seems to be disintegrating, and the primordial group members appear to be dispersing out rapidly. Searching for new open clusters in the vicinity of young or grouped OCs using Gaia data is an efficient strategy to find new associated OCs forming primordial groups. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.12613.md
    + _build/html/tmp_2211.12613/./J1120_bin_spec_corr_sky_zoomin.png
    + _build/html/tmp_2211.12613/./xhi_other_constrain_cut_misc_scale.png
    + _build/html/tmp_2211.12613/./J1120_flux_distribution.png
exported in  _build/html/2211.13146.md
    + _build/html/tmp_2211.13146/./posterimage.png
    + _build/html/tmp_2211.13146/./2_targetvsTNGintarget_Exclusion.png
    + _build/html/tmp_2211.13146/./4_finalsample.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  Forests

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.12613-b31b1b.svg)](https://arxiv.org/abs/2211.12613)<mark>Appeared on: 2022-11-24</mark> - _13 pages, 6 figures, 2 tables, accepted for publication in ApJ_

</div>
<div id="authors">

Xiangyu Jin, et al. -- incl., <mark>Frederick B. Davies</mark>, <mark>Joseph F. Hennawi</mark>, <mark>Fabian Walter</mark>

</div>
<div id="abstract">

**Abstract:** Cosmic reionization was the last major phase transition of hydrogen from neutral to highly ionized in the intergalactic medium (IGM). Current observations show that the IGM is significantly neutral at$z>7$, and largely ionized by$z\sim5.5$. However, most methods to measure the IGM neutral fraction are highly model-dependent, and are limited to when the volume-averaged neutral fraction of the IGM is either relatively low ($\overline{x}_{\rm HI} \lesssim 10^{-3}$) or close to unity ($\overline{x}_{\rm HI}\sim 1$). In particular, the neutral fraction evolution of the IGM at the critical redshift range of$z=6-7$is poorly constrained. We present new constraints on$\overline{x}_{\rm HI}$at$z\sim5.1-6.8$, by analyzing deep optical spectra of$53$quasars at$5.73<z<7.09$.We derive model-independent upper limits on the neutral hydrogen fraction based on the fraction of “dark” pixels identified in the Lyman$\alpha$(Ly$\alpha$) and Lyman$\beta$(Ly$\beta$) forests, without any assumptions on the IGM model or the intrinsic shape of the quasar continuum. They are the first model-independent constraints on the IGM neutral hydrogen fraction at$z\sim6.2-6.8$using quasar absorption measurements. Our results give upper limits of$\overline{x}_{\rm HI}(z=6.3) < 0.79\pm0.04$(1$\sigma$),$\overline{x}_{\rm HI} (z=6.5) < 0.87\pm0.03$(1$\sigma$), and$\overline{x}_{\rm HI} (z=6.7) < 0.94^{+0.06}_{-0.09}$(1$\sigma$). The dark pixel fractions at$z>6.1$are consistent with the redshift evolution of the neutral fraction of the IGM derived from the Planck 2018.

</div>

<div id="div_fig1">

<img src="tmp_2211.12613/./J1120_bin_spec_corr_sky_zoomin.png" alt="Fig2" width="100%"/>

**Figure 2. -** J1120+0641 binned spectrum in the Ly$\alpha$ forests (upper panel), Ly$\beta$ forests (lower left panel), and the redshift overlapping regions in Ly$\alpha$ and Ly$\beta$ forests (lower middle panel). The size of each binned pixel is $3.3 $cMpc. The binned $2\sigma$ spectral uncertainty is shown as the red line. The binned flux density is presented in bars, and identified "dark" pixels are shown in black. The sky OH emission is shown by the blue line \citep{Rousselot2000AA}, and our spectral binning method effectively puts the majority of pixels between sky OH emission lines. Yellow hatched pixels are manually masked during our analysis, as those fluxes are plausibly from residual sky OH emission. The zero flux level is corrected by the $2\sigma$ clipping median of all pixels in the Ly$\alpha$ forest (i. e., the vertical dashed line shown in Figure \ref{fig:1120_flux_distribution}).
     (*fig:binned_spec_correction*)

</div>
<div id="div_fig2">

<img src="tmp_2211.12613/./xhi_other_constrain_cut_misc_scale.png" alt="Fig4" width="100%"/>

**Figure 4. -** Constraints on the IGM neutral hydrogen fraction $\overline{x}_{\rm HI}$ from high-$z$ quasars studies and Planck 2018 results. The upper limits on $\overline{x}_{\rm HI}$ from dark pixels are in red (this work, derived from flux-corrected $\tau^{\alpha}_{\rm lim,2\sigma}>2.5$ pixels at $z>6$), magenta (this work, derived from flux-corrected $\tau^{\alpha}_{\rm lim,2\sigma}>4.5$ pixels at $z<6$), and blue \citep{McGreer2015MNRAS}. Constraints on $\overline{x}_{\rm HI}$ derived from Ly$\alpha$ effective optical depth are shown by black squares \citep{Fan2006AJ}, green squares \citep{Yang2020ApJb}, and yellow squares \citep{Bosman2021MNRAS}. The upper limits on $\overline{x}_{\rm HI}$, inferred from long dark gap length distributions in Ly$\alpha$ and Ly$\beta$ forests, are shown in blue pentagons \citep{Zhu2022ApJ}. At $z>7$, individual measurements on the neutral hydrogen fraction from quasar damping wings are denoted by hexagons \citep{Greig2017MNRAS,Banados2018Natur,Davies2018ApJ,Greig2019MNRAS,Wang2020ApJ,Yang2020ApJa}. The $1\sigma$ and $2\sigma$ reionization history derived from Planck 2018 results by assuming the FlexKnot model are denoted by the dark grey shaded region and the light grey shaded region \citep{Planck2020AA}. The colored regions display $1\sigma$ reionization histories in \citealt[][]{Robertson2015ApJ}(red), \citealt{Finkelstein2019ApJ}(blue), and \citealt{Naidu2020ApJ}(purple). (*fig:nhi_other_constraints*)

</div>
<div id="div_fig3">

<img src="tmp_2211.12613/./J1120_flux_distribution.png" alt="Fig6" width="100%"/>

**Figure 6. -** To correct "zero" flux level in VLT/X-Shooter spectra, we apply the median flux derived from sigma clipped pixels to the VLT/X-Shooter spectra. The original flux distribution of pixels in the Ly$\alpha$ forest from the J1120+0641 VLT/X-Shooter spectrum is shown by the black histogram. The median derived from the sigma clipping is denoted by the vertical black dashed line, and the 2$\sigma$ range from sigma clipping is in the grey shaded region. The flux distribution corrected by the sigma-clipped median is shown by the red histogram. (*fig:1120_flux_distribution*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# iMaNGA: mock MaNGA galaxies based on IllustrisTNG-50 and MaStar SSPs. II. the catalogue

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.13146-b31b1b.svg)](https://arxiv.org/abs/2211.13146)<mark>Appeared on: 2022-11-24</mark> - _arXiv admin note: text overlap with arXiv:2203.11575_

</div>
<div id="authors">

Lorenza Nanni, et al. -- incl., <mark><mark>Justus Neumann</mark></mark>, <mark><mark>Lewis Hill</mark></mark>, <mark><mark>Annalisa Pillepich</mark></mark>

</div>
<div id="abstract">

**Abstract:** To probe the current theory on galaxy formation and evolution, an increased synergy between observations and simulations is necessary. For this reason, in our previous paper of this series, we presented a method to mock  SDSS-IV/MaNGA integral-field spectroscopic galaxy observations from cosmological simulations of galaxy formation. Here we present the resulting mock galaxy catalogue.  This catalogue consists of 1,000 unique galaxies in IllustrisTNG-50 falling into the SDSS-IV/MaNGA-primary target footprint, defined in the redshift and i-band absolute magnitude space.In this paper, we describe the general characteristics of the catalogue, in terms of morphology, kinematics, and stellar population properties. We also investigate our ability to recover the galaxy characteristics, as given by the simulations, analysing the synthetic spectra. We show that the `intrinsic' and recovered kinematics, age and metallicity are consistent within 1${\sigma}$, with residuals over all tassels ($\sim 8$million) consistent with$0$at the$68\%$confidence level.  We also compare `intrinsic' and recovered star formation histories, finding a close resemblance. Therefore, our mocking and spectral fitting processes do not distort intrinsic galaxy properties, hence we can use these results for scientific analysis. In the next papers of this series, we will present a comprehensive comparison and scientific analysis of TNG50 simulations with MaNGA observational results.

</div>

<div id="div_fig1">

<img src="tmp_2211.13146/./posterimage.png" alt="Fig15" width="100%"/>

**Figure 15. -** RGB images of 8 galaxies in the TNG50 iMaNGA sample (see \cref{sec:mangasample}). The RGB images are constructed from their mock MaNGA-like datacubes, generated with the method presented in \citet[][]{Nanni2022}. Different sky-sizes and morphologies can be appreciated. The redshift and the ID of the galaxies in TNG50 are reported in the upper-left corner for each galaxy. Their key properties are stated in Table \ref{tab:prop}.
     (*fig:posterimage*)

</div>
<div id="div_fig2">

<img src="tmp_2211.13146/./2_targetvsTNGintarget_Exclusion.png" alt="Fig3" width="100%"/>

**Figure 3. -** LoF entryThe distribution of the TNG50 galaxies in the initial sample (see \cref{sec:tnggal}) falling into the MaNGA-primary target footprint, defined in the redshift and i-band AB magnitude space (upper panel), and their distribution in i-band AB magnitude (bottom panel). There are 3,152 galaxies in this sample, which we refer to as "parent sample". \textit{Upper panel:} The distribution of the TNG50 galaxies in the parent sample in the magnitude-redshift space, colour-coded by the number of galaxies, and the MaNGA-primary sample footprint (black solid line). The grey crosses represent the galaxies excluded from the sample because not in the MaNGA-primary target. (*fig:targetvsTNGintarget*)

</div>
<div id="div_fig3">

<img src="tmp_2211.13146/./4_finalsample.png" alt="Fig5" width="100%"/>

**Figure 5. -** LoF entryThe distribution of TNG50 galaxies in the iMaNGA sample in the redshift and i-band AB magnitude space (upper panel) and their distribution in i-band AB magnitude in comparison to the previous defined samples, i.e. the initial and parent samples (bottom panel). The iMaNGA sample is built from the parent sample randomly extracting 1000 galaxies from it, with a higher probability to get selected if in magnitude bins less populated. \textit{Upper panel:} the distribution of the TNG50  galaxies in the iMaNGA sample in magnitude-redshift space, colour-coded by the number of galaxies. The black solid line reports the MaNGA-primary sample footprint.  \textit{Bottom panel:} the density distribution in i-band  AB $M_i-5\log_{10}h$ of the samples of TNG50 galaxies discussed so far, i.e. the initial sample (black hatch-filled histogram),  the parent sample (yellow histogram) and  the iMaNGA sample (teal empty histogram).
     (*fig:finalsample*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

98  publications in the last 7 days.
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers